In [ ]:
%load_ext autoreload
%autoreload
from factory import *
import torch
#from model import Unet
from catalyst.dl.callbacks import CriterionCallback, EarlyStoppingCallback
from catalyst.dl.runner import SupervisedRunner
from pytorch_toolbelt import losses as L
import collections
from pytorch_toolbelt.utils.catalyst import * 
from metrics import *
import custom_tta as tta
from pytorch_toolbelt.inference.tiles import *
import matplotlib.pyplot as plt
from viz_utils import *
from tqdm import tqdm
import segmentation_models_pytorch as smp
from tqdm import tqdm_notebook
import pretrainedmodels
from pytorch_toolbelt.inference import tta
from itertools import combinations
from catalyst import utils
import seaborn as sns
%matplotlib inline

In [ ]:
all_encoder_names = ['efficientnet-b3',
            'efficientnet-b5',
            'efficientnet-b7',
            'resnext50_32x4d',
            'se_resnext50_32x4d',
            'se_resnet50',
            'se_resnext101_32x4d', 
            'resnext50_32x4d',
            'se_resnext50_32x4d',
            'se_resnext101_32x4d',
            'efficientnet-b5',
            'resnext101_32x8d']
all_experiment_names = ['efficientnet-b3_simple',
            'efficientnet-b5_simple',
            'efficientnet-b7_simple',
            'resnext50_32x4d_simple',
            'se_resnext50_32x4d_simple',
            'se_resnet50_simple',
            'se_resnext101_32x4d_simple', 
            'resnext50_32x4d_with_mask_and_boundaries',
            'se_resnext50_32x4d_with_mask_and_boundaries',
            'se_resnext101_32x4d_with_mask_and_boundaries',
            'efficientnet-b5_with_mask_and_boundaries',
            'resnext101_32x8d_with_mask_and_boundaries']
all_log_dirs = ['logs/efficientnet-b3_simple',
            'logs/efficientnet-b5_simple',
            'logs/efficientnet-b7_simple',
            'logs/resnext50_32x4d_simple',
            'logs/se_resnext50_32x4d_simple',
            'logs/se_resnet50_simple',
            'logs/se_resnext101_32x4d_simple', 
            'logs/resnext50_32x4d_with_mask_and_boundaries',
            'logs/se_resnext50_32x4d_with_mask_and_boundaries',
            'logs/se_resnext101_32x4d_with_mask_and_boundaries',
            'logs/efficientnet-b5_with_mask_and_boundaries',
            'logs/resnext101_32x8d_with_mask_and_boundaries']
output_channels = [4,4,4,4,4,4,4,6,6,6,6,6]
train_df_path = 'data/train.csv'
data_folder = "data/train_images/"
test_data_folder = "data/test_images/"
val_output_folder = "data/validation_predictions"
sample_submission_path = 'data/sample_submission.csv'

In [ ]:
train_df, val_df = return_masks(train_df_path)

In [ ]:
assert(len(all_encoder_names)==len(all_log_dirs))
assert(len(all_encoder_names)==len(output_channels))

In [ ]:
len(all_encoder_names)

In [ ]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, fnames, transforms):
        self.root = root
        self.fnames = fnames.tolist()
        self.num_samples = len(self.fnames)
        self.transforms = transforms

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        image = self.transforms(image=image)["image"]
        image = torch.from_numpy(image.transpose((2, 0, 1))).float()
        return {'features':image}

    def __len__(self):
        return self.num_samples

In [ ]:
gt_masks = []
images_id = []
for image_idx in tqdm(range(len(val_df.index.values))):
    image_name =  val_df.index.values[image_idx]
    labels = val_df.loc[image_name,:][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32) # float32 is V.Imp
    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 1
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    gt_masks.append(masks)
    images_id.append(val_df.index.values[image_idx])

In [ ]:
dict_of_gt_masks = dict(zip(images_id, gt_masks))

In [ ]:
device = utils.get_device()
print(f"device: {device}")

In [ ]:
val_loader = DataLoader(
    TestDataset(data_folder, val_df.index.values, validation_augmentations()),
    batch_size=32,
    num_workers=4,
    shuffle=False,   
    )

In [ ]:
runner = SupervisedRunner(device=device, input_key="features")
for idx, encoder_name in enumerate(all_encoder_names):
    print('Infering {}'.format(encoder_name))
    runner = SupervisedRunner(device=device, input_key='features')
    model = smp.Unet(classes=output_channels[idx], 
                    encoder_name=encoder_name,
                    encoder_weights=None)
    predictions = runner.predict_loader(
            model=model,
            loader=val_loader,
            resume=f"{all_log_dirs[idx]}/checkpoints/best.pth",
            verbose=True,
        ) 
    np.save(os.path.join(val_output_folder, all_experiment_names[idx]+'.npz'), predictions)
    del model

Now load and find validation score for each 1,2,3 models

In [ ]:
min_area = [800, 800, 2000, 3500]
thr_prediction = 0.5
dices_single_model = []
models = []
for exp in combinations(all_experiment_names,1):
    data = np.load(os.path.join(val_output_folder, exp[0]+'.npz.npy'))
    data = dict(zip(images_id, data))
    print('{} loaded'.format(exp[0]))
    dice_preds = []
    images_per_defect = []
    for image_name in tqdm(images_id, total=len(images_id)):
        with torch.no_grad():
            predictions = nn.Sigmoid()(torch.from_numpy(data[image_name])).numpy()
        for defect_type in range(4):
            mask_pred =  (predictions[defect_type,...] > thr_prediction).astype(int)
            mask_gt = dict_of_gt_masks[image_name][...,defect_type]
            if mask_pred.sum() < min_area[defect_type]:
                mask_pred = np.zeros(mask_pred.shape)
            dice_gt_pr = dice(mask_gt,
                              mask_pred,
                              empty_score=1.0)
            dice_preds.append(dice_gt_pr)
            images_per_defect.append(images_id[idx])
    print('{} : DICE validation {}'.format(exp[0], np.mean(dice_preds)))   
    del data
    dices_single_model.append(np.mean(dice_preds))
    models.append(exp[0])

In [ ]:
single_models_score = pd.DataFrame({'models':models,'score':dices_single_model})
single_models_score = single_models_score.sort_values(['score'],ascending=False)
single_models_score.head()

In [ ]:
selected_models = single_models_score.models.values[single_models_score.score.values>=0.935]

In [ ]:
selected_models

In [ ]:
min_area = [600, 600, 1000, 2000]
thr_prediction = 0.5
dices_scores = []
models = []
for exp in combinations(selected_models, 5):
    data = [np.load(os.path.join(val_output_folder, exp_+'.npz.npy')) for exp_ in exp]
    data = [dict(zip(images_id, data_)) for data_ in data]
    print('{} loaded'.format(' '.join(exp)))
    dice_preds = []
    images_per_defect = []
    for image_name in tqdm(images_id, total=len(images_id)):
        with torch.no_grad():
            predictions = [nn.Sigmoid()(torch.from_numpy(data_[image_name][:4,...])).numpy() for data_ in data]
            predictions = np.stack(predictions).mean(axis=0)
        for defect_type in range(4):
            mask_pred =  (predictions[defect_type,...] > thr_prediction).astype(int)
            mask_gt = dict_of_gt_masks[image_name][...,defect_type]
            if mask_pred.sum() < min_area[defect_type]:
                mask_pred = np.zeros(mask_pred.shape)
            dice_gt_pr = dice(mask_gt,
                              mask_pred,
                              empty_score=1.0)
            dice_preds.append(dice_gt_pr)
            images_per_defect.append(images_id[idx])
    print('{} : DICE validation {}'.format(' '.join(exp), np.mean(dice_preds)))   
    del data
    dices_scores.append(np.mean(dice_preds))
    models.append(' '.join(exp))

In [ ]:
five_models_score = pd.DataFrame({'models':models,'score':dices_scores})
five_models_score = five_models_score.sort_values(['score'],ascending=False)
five_models_score.head()

In [ ]:
min_area = [600, 600, 1000, 2000]
thr_prediction = 0.5
dices_scores = []
models = []
for exp in combinations(selected_models, 6):
    data = [np.load(os.path.join(val_output_folder, exp_+'.npz.npy')) for exp_ in exp]
    data = [dict(zip(images_id, data_)) for data_ in data]
    print('{} loaded'.format(' '.join(exp)))
    dice_preds = []
    images_per_defect = []
    for image_name in tqdm(images_id, total=len(images_id)):
        with torch.no_grad():
            predictions = [nn.Sigmoid()(torch.from_numpy(data_[image_name][:4,...])).numpy() for data_ in data]
            predictions = np.stack(predictions).mean(axis=0)
        for defect_type in range(4):
            mask_pred =  (predictions[defect_type,...] > thr_prediction).astype(int)
            mask_gt = dict_of_gt_masks[image_name][...,defect_type]
            if mask_pred.sum() < min_area[defect_type]:
                mask_pred = np.zeros(mask_pred.shape)
            dice_gt_pr = dice(mask_gt,
                              mask_pred,
                              empty_score=1.0)
            dice_preds.append(dice_gt_pr)
            images_per_defect.append(images_id[idx])
    print('{} : DICE validation {}'.format(' '.join(exp), np.mean(dice_preds)))   
    del data
    dices_scores.append(np.mean(dice_preds))
    models.append(' '.join(exp))

In [ ]:
six_models_score = pd.DataFrame({'models':models,'score':dices_scores})
six_models_score = six_models_score.sort_values(['score'],ascending=False)
six_models_score.head()

In [ ]:
min_area = [600, 600, 1000, 2000]
thr_prediction = 0.5
dices_scores = []
models = []
for exp in combinations(selected_models, 4):
    data = [np.load(os.path.join(val_output_folder, exp_+'.npz.npy')) for exp_ in exp]
    data = [dict(zip(images_id, data_)) for data_ in data]
    print('{} loaded'.format(' '.join(exp)))
    dice_preds = []
    images_per_defect = []
    for image_name in tqdm(images_id, total=len(images_id)):
        with torch.no_grad():
            predictions = [nn.Sigmoid()(torch.from_numpy(data_[image_name][:4,...])).numpy() for data_ in data]
            predictions = np.stack(predictions).mean(axis=0)
        for defect_type in range(4):
            mask_pred =  (predictions[defect_type,...] > thr_prediction).astype(int)
            mask_gt = dict_of_gt_masks[image_name][...,defect_type]
            if mask_pred.sum() < min_area[defect_type]:
                mask_pred = np.zeros(mask_pred.shape)
            dice_gt_pr = dice(mask_gt,
                              mask_pred,
                              empty_score=1.0)
            dice_preds.append(dice_gt_pr)
            images_per_defect.append(images_id[idx])
    print('{} : DICE validation {}'.format(' '.join(exp), np.mean(dice_preds)))   
    del data
    dices_scores.append(np.mean(dice_preds))
    models.append(' '.join(exp))

In [ ]:
four_models_score = pd.DataFrame({'models':models,'score':dices_scores})
four_models_score = four_models_score.sort_values(['score'],ascending=False)
four_models_score.head()

In [ ]:
all_val_scores = pd.concat([five_models_score, single_models_score, four_models_score, single_models_score])
all_val_scores = all_val_scores.sort_values(['score'],ascending=False)
all_val_scores.loc[:,'N_models_in_stack'] = all_val_scores.models.apply(lambda x: len(x.split(' ')))
all_val_scores.head()

In [ ]:
all_val_scores.to_csv('./validation_results.tsv', index=False)

In [ ]:
print('The best composition of models is : {}, \nsecond best is : {},\n third : {}'.format(all_val_scores.models.values[0], 
                                                                                all_val_scores.models.values[1],
                                                                                all_val_scores.models.values[2]))

#### Find best thresholds for min_mask_size : tune different thesholds per each defect type

In [ ]:
tuning_sizes = [100*x for x in range(51)]

In [ ]:
all_ts = []
all_dice_score_ts = []
all_tested_models = []
all_defect_types = []
for models_idx in range(4):
    data = [np.load(os.path.join(val_output_folder, exp_+'.npz.npy')) for exp_ in all_val_scores.models.values[models_idx].split(' ')]
    print('{} are loaded'.format(all_val_scores.models.values[models_idx]))
    data = [dict(zip(images_id, data_)) for data_ in data]
    for defect_type in range(4):
        predictions = np.zeros((len(images_id), 256, 1600))
        mask_gt = np.zeros((len(images_id), 256, 1600))
        for idx in tqdm(range(len(images_id)), total=len(images_id)):
            image_name = images_id[idx]
            pred_ = [nn.Sigmoid()(torch.from_numpy(data_[image_name][defect_type,...])).numpy() for data_ in data]
            pred_ = (np.stack(pred_).mean(axis=0) > thr_prediction).astype(int)
            predictions[idx] = pred_
            mask_gt[idx] = dict_of_gt_masks[image_name][...,defect_type]
        mask_sizes = [x.sum() for x in predictions]
        predictions_masked = predictions.copy()
        best_score = 0
        for ts in tuning_sizes:
            predictions_masked[[x<ts for x in mask_sizes]] = np.zeros((256, 1600))
            dice_score_ts = np.mean([dice(pair[0], pair[1], empty_score=1.0) for pair in zip(mask_gt, predictions_masked)])
            print('{} : {}'.format(ts, dice_score_ts))
            if dice_score_ts > best_score:
                best_ts = ts
                best_score = dice_score_ts
            all_ts.append(ts)
            all_dice_score_ts.append(dice_score_ts)
            all_tested_models.append(all_val_scores.models.values[models_idx])
            all_defect_types.append('defect_type '+str(defect_type))
        print('{}, defect type {} : best score {} at {}'.format(all_val_scores.models.values[models_idx], defect_type, 
                                                               best_score, best_ts))
    del data

In [ ]:
validation_result = pd.DataFrame({'threshold':all_ts,
                                  'dice_score':all_dice_score_ts,
                                  'models':all_tested_models,
                                  'defect_type':all_defect_types})

In [ ]:
validation_result_best = validation_result.sort_values('dice_score', ascending=False).drop_duplicates(['defect_type','models'])
validation_result_best.head()

In [ ]:
f, ax = plt.subplots(1,1,figsize=(15,15))
validation_result = validation_result.loc[validation_result.threshold<4500,:]
sns.lineplot(x="threshold", y="dice_score",
                   style="models", hue="defect_type",
                  data=validation_result, ax=ax)
plt.tight_layout()

### Now recalculate using best threshold for each

In [ ]:
dices_scores = []
models = []
for models_idx in range(4):
    data = [np.load(os.path.join(val_output_folder, exp_+'.npz.npy')) for exp_ in all_val_scores.models.values[models_idx].split(' ')]
    data = [dict(zip(images_id, data_)) for data_ in data]
    print('{} loaded'.format(' '.join(exp)))
    dice_preds = []
    images_per_defect = []
    for image_name in tqdm(images_id, total=len(images_id)):
        with torch.no_grad():
            predictions = [nn.Sigmoid()(torch.from_numpy(data_[image_name][:4,...])).numpy() for data_ in data]
            predictions = np.stack(predictions).mean(axis=0)
        for defect_type in range(4):
            mask_pred =  (predictions[defect_type,...] > thr_prediction).astype(int)
            mask_gt = dict_of_gt_masks[image_name][...,defect_type]
            size_mask = (validation_result_best.models==all_val_scores.models.values[models_idx]) & (validation_result_best.defect_type=='defect_type '+str(defect_type))
            if mask_pred.sum() < validation_result_best.threshold[size_mask].values[0]:
                mask_pred = np.zeros(mask_pred.shape)
            dice_gt_pr = dice(mask_gt,
                              mask_pred,
                              empty_score=1.0)
            dice_preds.append(dice_gt_pr)
            images_per_defect.append(images_id[idx])
    print('{} : DICE validation {}'.format(all_val_scores.models.values[models_idx], np.mean(dice_preds)))   
    del data
    dices_scores.append(np.mean(dice_preds))
    models.append(all_val_scores.models.values[models_idx])

In [ ]:
final_dt = pd.DataFrame({'models':models, 'score':dices_scores})
final_dt = final_dt.sort_values(['score'],ascending=False)
final_dt.head()

Now get predictions for test

In [ ]:
sample_submission = pd.read_csv(sample_submission_path)
test_images = np.unique([x.split('_')[0] for x in sample_submission.ImageId_ClassId.values])
test_loader = DataLoader(
    TestDataset(test_data_folder, test_images, validation_augmentations()),
    batch_size=32,
    num_workers=4,
    shuffle=False,   
    )

In [ ]:
final_exps = final_dt.models.values[0].split(' ')

In [ ]:
runner = SupervisedRunner(device=device, input_key="features")
predictions_test = []
for exp_name in final_exps:
    print('Infering test using {}'.format(exp_name))
    idx = np.where(np.array(all_experiment_names)==final_exps[0])[0][0]
    runner = SupervisedRunner(device=device, input_key='features')
    model = smp.Unet(classes=output_channels[idx], 
                    encoder_name=all_encoder_names[idx],
                    encoder_weights=None)
    predictions_test.append(runner.predict_loader(
            model=model,
            loader=test_loader,
            resume=f"{all_log_dirs[idx]}/checkpoints/best.pth",
            verbose=True,
        ))

In [ ]:
images_id = []
masks_rle = []
for idx in tqdm(range(len(test_images))):
    image_pred = np.stack([nn.Sigmoid()(torch.from_numpy(data_[idx][:4,...])).numpy() for data_ in predictions_test]).mean(axis=0)
    for defect_type in range(4):
        image_pred_bin = image_pred[defect_type,...]>0.5
        ts_mask = (validation_result_best.models==final_dt.models.values[0]) & (validation_result_best.defect_type == 'defect_type '+str(defect_type))
        if image_pred_bin.sum() < validation_result_best.threshold[ts_mask].values[0]:
            masks_rle.append('')
        else:
            masks_rle.append(mask2rle(image_pred_bin))
        images_id.append(test_images[idx]+'_{}'.format(defect_type+1))     

In [ ]:
submission = pd.DataFrame({'ImageId_ClassId':images_id,'EncodedPixels':masks_rle})
submission.head()

#### Add classification predictions

In [ ]:
classifications = pd.read_csv('classificaion_test/effnetb3_3folds.csv')
classifications.columns.values[1] = 'has_mask'

In [ ]:
submission = submission.merge(classifications)

In [ ]:
submission.head()

In [ ]:
submission.loc[:,'has_mask_segmentations'] = submission.EncodedPixels.apply(lambda x: int(x!=''))

In [ ]:
submission.groupby(["has_mask", "has_mask_segmentations"]).size()

In [ ]:
submission.loc[submission.has_mask==0, 'EncodedPixels'] = ''

In [ ]:
submission.to_csv('./submission_vA.tsv',index=False)